In [155]:
import pandas as pd
import numpy as np 
from pandas_datareader import data
from func_pyeconomics import graf_plotly

In [156]:
ativos = ['CMIG4.SA','ELET3.SA','EGIE3.SA','EQTL3.SA','TAEE3.SA','TRPL4.SA','BOVA11.SA']

In [157]:
dataframe = pd.DataFrame()
for i in ativos:
    frame = data.DataReader(i, data_source='yahoo', start='2015-01-01', end='2022-01-01')
    frame = frame['Adj Close']
    frame = pd.DataFrame(frame)
    frame = frame.rename(columns={'Adj Close':f'{i}'})
    dataframe = pd.concat([frame, dataframe], axis = 1)

### Normalização da carteira

In [158]:
df_normalizado = pd.DataFrame()
for i in dataframe.columns[0:]:
    df_normalizado[i] = dataframe[i] / dataframe[i][0]
df_carteira = df_normalizado.drop(columns='BOVA11.SA')
benchmark = pd.DataFrame(df_normalizado['BOVA11.SA'])
benchmark

,BOVA11.SA
Date,
2015-01-02,1.000000
2015-01-05,0.980110
2015-01-06,0.985612
2015-01-07,1.018832
2015-01-08,1.026449
...,...
2021-04-20,2.443504
2021-04-22,2.432289
2021-04-23,2.453872


In [159]:
df_carteira['CARTEIRA'] = df_carteira.sum(axis=1) / len(df_carteira.columns)
carteira = pd.DataFrame(df_carteira['CARTEIRA'])

In [160]:
cart_bench = pd.concat([carteira, benchmark], axis = 1)

In [161]:
graf_plotly(cart_bench)